# 03 – Memory & LCEL Basics

**Learning Goals:**
- Understand conversational memory in LangChain
- Compare memory types: Buffer vs Summary
- Master LCEL (LangChain Expression Language) composition
- Build streaming, retry, and fallback patterns

**What we'll cover:**
1. **Memory 101** - Buffer and Summary memory patterns
2. **Memory in Chains** - Inject memory into conversational flows
3. **LCEL Basics** - Compose runnables with `|` operator
4. **Advanced LCEL** - Streaming, retry, fallbacks
5. **Tool Use (Optional)** - Simple tool integration

**Note:** This notebook focuses on fundamentals, not RAG. No ChromaDB or retrieval here.


In [ ]:
# ⚙️ Global Config & Services (using centralized modules)
import sys
import json
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv

# Add parent directory to path and change to project root
import os

# Get the notebook's current directory and find project root
notebook_dir = Path.cwd()
if notebook_dir.name == "notebooks":
    project_root = notebook_dir.parent
else:
    project_root = notebook_dir

# Change to project root and add to path
os.chdir(project_root)
sys.path.insert(0, str(project_root))

print(f"📂 Working directory: {os.getcwd()}")

from src.services.llm_services import (
    load_config,
    get_llm,
    validate_api_keys,
    print_config_summary
)

# Load environment variables
load_dotenv()

# Load configuration from config.yaml (now we're in project root)
config = load_config("src/config/config.yaml")

# Validate API keys
validate_api_keys(config, verbose=True)

# Print summary
print_config_summary(config)
print(f"  Note: Temperature is {config['temperature']} (good for conversational demos)")


✅ Config loaded:
  LLM: openrouter (openai/gpt-4o-mini)
  Embeddings: sbert / sentence-transformers/all-MiniLM-L6-v2
  Temperature: 0.2
  Artifacts: ./artifacts
  Note: Temperature is 0.2 (good for conversational demos)


/Users/machinelearningzuu/Dropbox/Zuu Crew/Courses/🚧 AI Engineer Essentials/Codes/RAG Systems/src/services/llm_services.py:338: UserWarning: ⚠️  GROQ_API_KEY not found in environment
  warnings.warn(f"⚠️  {key} not found in environment")
/Users/machinelearningzuu/Dropbox/Zuu Crew/Courses/🚧 AI Engineer Essentials/Codes/RAG Systems/src/services/llm_services.py:338: UserWarning: ⚠️  GOOGLE_API_KEY not found in environment
  warnings.warn(f"⚠️  {key} not found in environment")
/Users/machinelearningzuu/Dropbox/Zuu Crew/Courses/🚧 AI Engineer Essentials/Codes/RAG Systems/src/services/llm_services.py:338: UserWarning: ⚠️  COHERE_API_KEY not found in environment
  warnings.warn(f"⚠️  {key} not found in environment")


In [2]:
# Initialize LLM using factory from llm_services
llm = get_llm(config)
print(f"✅ LLM initialized: {config['llm_provider']} / {config['llm_model']}")

# Verify API key with test completion
print("\n🔍 Testing API connection...")
try:
    test_response = llm.invoke("Say 'API working!' if you can read this.")
    test_msg = test_response.content if hasattr(test_response, 'content') else str(test_response)
    print(f"✅ API key verified: {test_msg[:50]}")
except Exception as e:
    print(f"❌ API key test failed: {e}")
    print("⚠️  Please check your .env file and API key configuration.")


✅ LLM initialized: openrouter / gpt-4o-mini

🔍 Testing API connection...
✅ API key verified: API working!


---

## Section A: Memory 101

LangChain provides memory primitives to maintain conversational context across turns.

### 1. ConversationBufferMemory

Stores **full chat history** in memory. Simple but can grow large.


In [ ]:
from langchain.memory import ConversationBufferMemory

### START CODE HERE ### (≈ 18-20 lines)
# YOUR CODE HERE
# HINTS:
# 1. Create a ConversationBufferMemory instance with return_messages=True:
#    buffer_memory = ConversationBufferMemory(return_messages=True)
#
# 2. Simulate 3 conversation turns using buffer_memory.save_context():
#    Turn 1:
#      - input: {"input": "Hi, my name is Alice."}
#      - output: {"output": "Hello Alice! Nice to meet you."}
#    Turn 2:
#      - input: {"input": "What's my name?"}
#      - output: {"output": "Your name is Alice."}
#    Turn 3:
#      - input: {"input": "What's the capital of France?"}
#      - output: {"output": "The capital of France is Paris."}
#
# 3. Load and print the memory variables:
#    - Use buffer_memory.load_memory_variables({})
#    - Print the result with a label like "📝 Buffer Memory:"
#
# 4. Print the memory_variables attribute:
#    - Print buffer_memory.memory_variables with a label like "💾 Memory keys:"

raise NotImplementedError("Complete the ConversationBufferMemory exercise")
### END CODE HERE ###


📝 Buffer Memory:
{'history': [HumanMessage(content='Hi, my name is Alice.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Alice! Nice to meet you.', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Alice.', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's the capital of France?", additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={})]}

💾 Memory keys: ['history']


/var/folders/b5/fr4bwywx1hl34z7s66ljm40r0000gn/T/ipykernel_5324/1920384859.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  buffer_memory = ConversationBufferMemory(return_messages=True)


### 2. ConversationSummaryMemory

Instead of storing full history, **summarizes** past conversation using an LLM. Reduces token usage but may lose details.


In [4]:
from langchain.memory import ConversationSummaryMemory

# Create summary memory (requires LLM)
summary_memory = ConversationSummaryMemory(llm=llm, return_messages=True)

# Simulate same conversation
summary_memory.save_context(
    {"input": "Hi, my name is Alice."},
    {"output": "Hello Alice! Nice to meet you."}
)
summary_memory.save_context(
    {"input": "What's my name?"},
    {"output": "Your name is Alice."}
)
summary_memory.save_context(
    {"input": "What's the capital of France?"},
    {"output": "The capital of France is Paris."}
)

# View summarized history
print("📝 Summary Memory:")
print(summary_memory.load_memory_variables({}))
print(f"\n📊 Summary is more compact than full buffer")


/var/folders/b5/fr4bwywx1hl34z7s66ljm40r0000gn/T/ipykernel_5324/1045286892.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(llm=llm, return_messages=True)


📝 Summary Memory:
{'history': [SystemMessage(content='The human introduces herself as Alice. The AI responds by greeting Alice and expressing pleasure in meeting her, confirming her name. The human then asks about the capital of France, and the AI replies that it is Paris.', additional_kwargs={}, response_metadata={})]}

📊 Summary is more compact than full buffer


### Trade-offs: Buffer vs Summary

| Memory Type | Pros | Cons |
|-------------|------|------|
| **Buffer** | Full detail, no LLM calls | Grows unbounded, context limits |
| **Summary** | Compact, scalable | LLM calls needed, possible drift |

**When to use:**
- **Buffer**: Short conversations, need exact history
- **Summary**: Long conversations, want cost efficiency


---

## Section B: Memory in Chains

Let's inject memory into a simple conversational chain.


In [5]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Create a conversational chain with memory
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False,  # Set to True to see internal prompts
)

# Multi-turn conversation
print("🗨️  Conversational Chain with Memory\n")

response1 = conversation.predict(input="Hi, I'm Bob and I love Python programming.")
print(f"User: Hi, I'm Bob and I love Python programming.")
print(f"AI: {response1}\n")

response2 = conversation.predict(input="What's my name?")
print(f"User: What's my name?")
print(f"AI: {response2}\n")

response3 = conversation.predict(input="What do I love?")
print(f"User: What do I love?")
print(f"AI: {response3}\n")

# View memory
print("📝 Stored Memory:")
print(memory.load_memory_variables({}))


/var/folders/b5/fr4bwywx1hl34z7s66ljm40r0000gn/T/ipykernel_5324/2397125044.py:6: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


🗨️  Conversational Chain with Memory

User: Hi, I'm Bob and I love Python programming.
AI: Hello, Bob! It's great to meet you! Python is such a versatile and powerful programming language. What do you enjoy most about it? Are you working on any specific projects or using any particular libraries? There’s so much you can do with Python, from web development with frameworks like Django and Flask to data analysis with libraries like Pandas and NumPy.

User: What's my name?
AI: Your name is Bob! It's nice to chat with you. How did you get into Python programming?

User: What do I love?
AI: You love Python programming! It's fantastic to hear that. Python has so many applications and a vibrant community. What aspects of Python do you find most enjoyable? Is it the simplicity of the syntax, the vast array of libraries, or perhaps the community support?

📝 Stored Memory:
{'history': "Human: Hi, I'm Bob and I love Python programming.\nAI: Hello, Bob! It's great to meet you! Python is such a ver

### Resetting Memory

Between sessions, clear memory to start fresh.


In [6]:
# Clear memory
memory.clear()

response4 = conversation.predict(input="What's my name?")
print(f"After clearing memory:")
print(f"User: What's my name?")
print(f"AI: {response4}")
print(f"\n✅ Memory reset - AI no longer remembers Bob")


After clearing memory:
User: What's my name?
AI: I’m not sure what your name is since I don’t have access to that information. But I’d love to know it if you’d like to share! What do you like to be called?

✅ Memory reset - AI no longer remembers Bob


---

## Section C: LCEL (LangChain Expression Language) Basics

LCEL is a declarative way to compose LangChain components using the `|` operator.

### Core Concepts

1. **Runnable**: Base interface for all LCEL components
2. **Pipe (`|`)**: Chain runnables together
3. **RunnablePassthrough**: Pass data through unchanged
4. **RunnableMap**: Apply multiple operations in parallel

### Simple LCEL Chain

Let's build: `PromptTemplate | LLM | StrOutputParser`


In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Define prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer concisely."),
    ("human", "{question}")
])

# Build LCEL chain
chain = prompt | llm | StrOutputParser()

# Invoke
response = chain.invoke({"question": "What is eczema and how is it treated?"})
print("🔗 Simple LCEL Chain:")
print(f"Question: What is eczema and how is it treated?")
print(f"Answer: {response}")


🔗 Simple LCEL Chain:
Question: What is eczema and how is it treated?
Answer: Eczema, also known as atopic dermatitis, is a chronic inflammatory skin condition characterized by dry, itchy, and inflamed skin. It often occurs in flare-ups and can affect people of all ages.

**Treatment options include:**

1. **Moisturizers:** Regular use of emollients to keep the skin hydrated.
2. **Topical corticosteroids:** To reduce inflammation and itching during flare-ups.
3. **Topical calcineurin inhibitors:** Non-steroidal medications that help reduce inflammation.
4. **Antihistamines:** To alleviate itching, especially at night.
5. **Phototherapy:** Controlled exposure to ultraviolet light for severe cases.
6. **Systemic medications:** In severe cases, oral or injectable medications may be prescribed.
7. **Avoiding triggers:** Identifying and avoiding irritants or allergens that can worsen symptoms.

Consulting a healthcare provider for a personalized treatment plan is recommended.


### RunnablePassthrough & RunnableMap

Use `RunnablePassthrough` to pass input data and `RunnableMap` (via dict) for parallel operations.


In [ ]:
from langchain_core.runnables import RunnableParallel

### START CODE HERE ### (≈ 20-25 lines)
# YOUR CODE HERE
# HINTS:
# 1. Create a prompt template using ChatPromptTemplate.from_template():
#    - The template should have placeholders: {context} and {question}
#    - Example format:
#      """Use the context to answer the question.
#      
#      Context: {context}
#      Question: {question}
#      
#      Answer:"""
#    - Store in variable: context_prompt
#
# 2. Build an LCEL chain using the | (pipe) operator:
#    chain_with_context = (
#        RunnableParallel({
#            "context": RunnablePassthrough(),   # Pass context through
#            "question": RunnablePassthrough(),  # Pass question through
#        })
#        | context_prompt    # Format the prompt
#        | llm               # Generate answer
#        | StrOutputParser() # Extract string from response
#    )
#
# 3. Test the chain by invoking with:
#    result = chain_with_context.invoke({
#        "context": "Machine learning is a field of AI that learns from data.",
#        "question": "What is ML?"
#    })
#
# 4. Print the result with label "🔗 Chain with Context:"

raise NotImplementedError("Complete the LCEL chain with context exercise")
### END CODE HERE ###


🔗 Chain with Context:
Answer: Machine learning (ML) is a field of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to learn from and make predictions or decisions based on data.


---

## Section D: Advanced LCEL Patterns

### 1. Streaming

Stream tokens as they're generated for better UX.


In [ ]:
import sys

print("🌊 Streaming response:")
print("Answer: ", end="")

### START CODE HERE ### (≈ 5-7 lines)
# YOUR CODE HERE
# HINTS:
# 1. Use chain.stream() with input: {"question": "Explain RAG in one sentence."}
# 2. Loop through the stream using: for chunk in chain.stream(...):
# 3. Print each chunk without newline: print(chunk, end="", flush=True)
#    - end="" prevents adding newline after each chunk
#    - flush=True ensures immediate output
# 4. After the loop, print two newlines and "✅ Streaming complete"

raise NotImplementedError("Complete the streaming exercise")
### END CODE HERE ###


🌊 Streaming response:
Answer: RAG (Retrieval-Augmented Generation) is a machine learning approach that combines retrieval of relevant documents from a knowledge base with generative models to produce more accurate and contextually relevant responses.

✅ Streaming complete


### 2. Retry with Fallback

Use `.with_retry()` for automatic retries and `.with_fallbacks()` for fallback models.


In [10]:
# Add retry logic (max 3 attempts)
chain_with_retry = chain.with_retry(stop_after_attempt=3)

print("🔄 Chain with retry enabled")
print("If LLM fails, will retry up to 3 times")

# Fallback example (requires a second LLM)
# Uncomment if you have multiple providers configured
# fallback_llm = ChatOpenAI(model="gpt-3.5-turbo")
# chain_with_fallback = chain.with_fallbacks([fallback_chain])
# print("✅ Fallback chain: tries primary LLM, falls back to gpt-3.5-turbo")

print("✅ Retry pattern configured")


🔄 Chain with retry enabled
If LLM fails, will retry up to 3 times
✅ Retry pattern configured


---

## Section E: Minimal Tool Use (Optional)

LCEL can invoke tools. Here's a simple example with a current time tool.


In [ ]:
from langchain.tools import tool
from langchain_core.runnables import RunnableLambda

### START CODE HERE ### (≈ 25-30 lines)
# YOUR CODE HERE
# HINTS:
# Part 1: Create a tool
# 1. Use the @tool decorator
# 2. Define function: def get_current_time() -> str:
# 3. Add docstring: """Returns the current time in ISO format."""
# 4. Import datetime: from datetime import datetime
# 5. Return: datetime.now().isoformat()
#
# Part 2: Test the tool
# 6. Print "🔧 Tool Demo:"
# 7. Invoke the tool: get_current_time.invoke({})
# 8. Print the result with label "Current time:"
#
# Part 3: Build a tool chain
# 9. Create a ChatPromptTemplate with:
#    - Placeholders: {question} and {tool_result}
#    - Instructions mentioning the tool
#    - Store in: tool_prompt
#
# 10. Build an LCEL chain:
#     tool_chain = (
#         RunnableParallel({
#             "question": RunnablePassthrough(),
#             "tool_result": RunnableLambda(lambda x: get_current_time.invoke({}))
#         })
#         | tool_prompt
#         | llm
#         | StrOutputParser()
#     )
#
# 11. Test with: tool_chain.invoke({"question": "What time is it?"})
# 12. Print the answer with label "🔗 Tool Chain Result:"

raise NotImplementedError("Complete the tool integration exercise")
### END CODE HERE ###


🔧 Tool Demo:
Current time: 2026-01-11T08:33:26.464479

🔗 Tool Chain Result:
Answer: The time is 08:33 on January 11, 2026.


---

## Save Manifest


In [12]:
manifests_dir = Path(config["artifacts_root"]) / "manifests"
manifests_dir.mkdir(parents=True, exist_ok=True)

manifest = {
    "notebook": "12_memory_lcel_basics",
    "topics": [
        "ConversationBufferMemory",
        "ConversationSummaryMemory",
        "ConversationChain with memory",
        "LCEL composition",
        "Streaming",
        "Retry patterns",
        "Tool use"
    ],
    "llm_provider": config["llm_provider"],
    "llm_model": config["llm_model"],
    "created_at": datetime.now().isoformat(),
}

manifest_path = manifests_dir / "memory_lcel.json"
with open(manifest_path, "w") as f:
    json.dump(manifest, f, indent=2)

print(f"✅ Manifest saved: {manifest_path}")


✅ Manifest saved: artifacts/manifests/memory_lcel.json


---

## Summary

**What we learned:**

### Memory
- ✅ **Buffer Memory**: Stores full history (simple but grows)
- ✅ **Summary Memory**: LLM-summarized history (compact but may drift)
- ✅ **Memory in Chains**: Inject context into conversational flows
- ✅ **Reset/Clear**: Start fresh between sessions

### LCEL
- ✅ **Composition**: Use `|` to chain runnables
- ✅ **RunnablePassthrough**: Pass data unchanged
- ✅ **RunnableParallel**: Run operations in parallel
- ✅ **Streaming**: Token-by-token generation
- ✅ **Retry**: Automatic retries on failure
- ✅ **Fallbacks**: Switch to backup LLM
- ✅ **Tools**: Integrate external functions

**Key Patterns:**
```python
# Simple chain
chain = prompt | llm | parser

# With context
chain = RunnableParallel({...}) | prompt | llm | parser

# With retry
chain = chain.with_retry(stop_after_attempt=3)

# With streaming
for chunk in chain.stream(input):
    print(chunk)
```

**Artifacts:**
- `./artifacts/manifests/memory_lcel.json`
